In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
### IMPORTING FILES FROM THE PROJECT ROOT.

import numpy as np
import plotly.graph_objects as go

#### --- MAKING ROOT PROJECT FOLDER VISIBLE AT SCRIPT LEVEL ---

##
## --- Finding root project folder ---
##     (needed to import settings)
##
import os, sys
def find_project_root(beacon="ROOT_BEACON"):
    current_path = os.getcwd()
    
    while True:
        if os.path.isfile(os.path.join(current_path, beacon)):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:
            raise RuntimeError(f"Unable to find project root.\nNo '{beacon}' file found in any parent directory.")
        
        current_path = parent_path

# --- Find and set project root ---
project_root = find_project_root()


## --- Adding root porject folder to current system path ---
if project_root not in sys.path:
    sys.path.insert(1, project_root)
    
####                    --- end ---


#### IMPORTING FROM ROOT PROJECT FOLDER 
from Utils import ArraySettings, SortingSettings
from Utils import ArrayDataManager
from Utils import ArrayStorageCompressor

## --- Updating the execution times storage folder ---
if project_root not in SortingSettings.EXECUTION_TIMES_FOLDER:
    SortingSettings.EXECUTION_TIMES_FOLDER = os.sep.join([project_root, SortingSettings.EXECUTION_TIMES_FOLDER])
    SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER = os.sep.join([project_root, SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER])

    assert os.path.exists(SortingSettings.EXECUTION_TIMES_FOLDER), f"Execution times folder does not exists. \nCurrent location set: {SortingSettings.EXECUTION_TIMES_FOLDER}"

    EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC = os.sep.join([SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER, "logaritmic"])
    EXECUTION_TIMES_GRAPH_FOLDER_LINEAR = os.sep.join([SortingSettings.EXECUTION_TIMES_GRAPH_FOLDER, "linear"])
    
    if not os.path.exists(EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC):
        os.makedirs(EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC)

    if not os.path.exists(EXECUTION_TIMES_GRAPH_FOLDER_LINEAR):
        os.makedirs(EXECUTION_TIMES_GRAPH_FOLDER_LINEAR)
    


### LOADING DATA FROM TIME FOLDER

In [ ]:
SortingSettings.EXECUTION_TIMES_FOLDER

In [ ]:
arrayTimeManager = ArrayDataManager.ExecutionTimeDataStorage()



for file in os.listdir(SortingSettings.EXECUTION_TIMES_FOLDER):
    if SortingSettings.EXECUTION_TIMES_FILE_EXTENSION in file:
        
        full_path_file = os.path.join(SortingSettings.EXECUTION_TIMES_FOLDER, file)
        algorithm_name = file[0:file.find(SortingSettings.EXECUTION_TIMES_FILE_EXTENSION)]
        read_execution_time_storage = ArrayStorageCompressor.readFromFile(full_path_file)
        assert len(read_execution_time_storage.keys()) == 1, f"Expected only one algorithm key in file {full_path_file}, got {read_execution_time_storage.keys()}"
        assert algorithm_name in read_execution_time_storage, f"Expected {algorithm_name} as key in file {full_path_file}, got {read_execution_time_storage.keys()}"

        arrayTimeManager.merge(read_execution_time_storage)
        


In [ ]:
arrayTimeManager.get_folders_intersection(["QuickSort3Way"])

## DEFINITION OF FUNCTIONS USED TO SHOW TIME DATA

### FUNCTION TO CALCULATE PLOT POINTS USING ARRAY MANAGER LOADED

In [ ]:
def calculatePoints(timeManager):
    length_values = [] ## x-values
    time_values = [] ## y-values
    error_values = [] ## error-values
    
    for arrayLength in timeManager.getKeys(toSort = True):
        executionTimeData = timeManager.get(arrayLength)
    
        executionTimeMean = np.mean(executionTimeData)
    
        executionTimeData_2 = np.mean(np.pow(executionTimeData, 2))
        executionTimeMean_2 = np.pow(executionTimeMean, 2)
    
        executionTimeVariance = executionTimeData_2 - executionTimeMean_2
        
        time_values.append(executionTimeMean)
        length_values.append(arrayLength)
        error_values.append(np.sqrt(executionTimeVariance))

    return length_values, time_values, error_values

### DEFINITION OF FUNCTION USED TO COMPUTE THE PLOTTING DATA

In [ ]:
arrayTimeManager["QuickSort"][list(arrayTimeManager["QuickSort"].keys())[0]]

In [ ]:
def computePlotData(algorithmName):
    traces = []
    for storage_path, multiprocessing_measurements in arrayTimeManager[algorithmName].items():
        storage_name = storage_path[storage_path.rfind(os.sep):]
        for processes_measurement in multiprocessing_measurements:
            array_chunks, processes = processes_measurement.to_tuple()
            x = []
            y = []
            err_plus = []
            err_minus = []
            for execution_time in processes_measurement.execution_times:
                variability_value = execution_time.get_variability()
                time_analysis = execution_time.get_time_analysis()
                mean = time_analysis.mean
                q01, q99 = time_analysis.quantiles[0.01], time_analysis.quantiles[0.99]
                x.append(variability_value)
                y.append(mean)
                err_plus.append(q99 - mean)
                err_minus.append(mean - q01)
            trace_name = f"{storage_path[storage_path.rfind(os.sep):]}\n({array_chunks} chunk, {processes} proc)"
            traces.append(go.Scatter(
                                x = x,
                                y = y,
                                text=[f"({array_chunks} chunk, {processes} proc)"] * len(x),
                                color = algorithmName,
                
                                error_y = dict(
                                    type = 'data',
                                    symmetric=False,
                                    array = err_plus,
                                    arrayminus = err_minus,
                                    visible = True),
                                name = algorithmName,
                                legendgroup= trace_name,
                                legendgrouptitle_text= trace_name,
                                line_dash = storage_name,
                                symbol =f"({array_chunks} chunk, {processes} proc)",
                                markers = True,
                                #marker=dict(size=6, symbol="circle", line=dict(width=1, color="DarkSlateGrey")),
                                mode = "lines+markers",
                                hovertemplate =
                                                '%{y:.4s}s ± (%{error_y.array:.4s}, %{error_y.arrayminus:.4s})s %{text}' +
                                                '<extra></extra>' ## 'extra' keyword removes trace category   
                            )
                         )
    return traces

### DEFINITION OF THE FUNCTION USED TO CREATE THE GRAPH LAYOUT

In [ ]:
def createLayout(log= False):

    axis_type = "linear" if not log else "log"
    
    return go.Layout(
        height = 700,
        title = dict(
            text = "Execution Times",
            font = dict(
                size= 30
            ),
            subtitle=dict(
                text = "Run times grouped by sorting algorithm showed in a {type}-{type} graph.".format(type= axis_type),
                font = dict(color="dimgrey", size=16)
            )
        ),
    
        xaxis = dict(
            title = dict(
                text = ("Array length" if ArraySettings.VARIABILITY == ArraySettings.Variability.onLength else "Different numbers") + " <i>{type}</i>".format(type= "n" if not log else "log(n)"),
                font=dict(size = 18)
            ),
            type = axis_type,
            hoverformat = ".5s"
        ),
        
        yaxis = dict(
            title = dict(
                text = "Execution time <i>{type}</i>".format(type= "s" if not log else "log(s)"),
                font=dict(size = 18)
            ),
            type = axis_type,
        ),
        
        legend = dict(
            title = dict(
                text = "Algorithms"
            ),
            font=dict(size = 14),
            groupclick='toggleitem',
            itemdoubleclick='toggleothers'
        ),
        
        font=dict(
            family="Georgia",
            color = "black",
            size=18,
        ),
        
        hoverlabel = dict(
            font = dict(
                size = 14,
                family = "monospace"
            )
        ),
        
        hovermode="x unified"
    )

### PLOT EXECUTION POINTS GROUPED BY SORTING ALGORITHM

### COLLECTING TIMING DATA FROM TIME MANAGER

In [ ]:
list(data.values())[-2][0]

In [ ]:

data = {}
layout = createLayout(log = False)
fig = go.Figure(layout = layout)
for algorithmName in list(arrayTimeManager.keys()):
    data.update({algorithmName: computePlotData(algorithmName)})
    for trace in data[algorithmName]:
        fig.add_trace(trace)


fig.show()

### WRITE ON FILE

In [ ]:
algorithmNames = list(arrayTimeManager.keys())
for b in [False, True]:
    layout = createLayout(log = b)
    
    for i, algorithmName in enumerate(algorithmNames):
        for e in range(i, len(algorithmNames)):
            pair_data = [data[algorithmNames[i]]]
            if i != e:
                fileName = f"{algorithmName}__vs__{algorithmNames[e]}"
                pair_data.append(data[algorithmNames[e]])
            else:
                fileName = f"{algorithmName}"
            fileName += ".svg"
            
            fig = go.Figure(data=pair_data, layout = layout)
            fig.write_image(os.sep.join([EXECUTION_TIMES_GRAPH_FOLDER_LINEAR if not b else EXECUTION_TIMES_GRAPH_FOLDER_LOGARITMIC, fileName]), width=1500, height=900, scale= 4)
    